<a href="https://colab.research.google.com/github/catasaurus/random-ai-notebooks/blob/main/hobs_bart_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
from transformers import TFAutoModel, BartTokenizer, DefaultDataCollator
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.7 MB/s 
     |████████████████████████████████| 101 kB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 33.4 MB/s 
     |████████████████████████████████| 596 kB 30.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 67.6 MB/s 
     |████████████████████████████████| 1.1 MB 61.3 MB/s 
     |████████████████████████████████| 140 kB 75.1 MB/s 
     |████████████████████████████████| 127 kB 69.4 MB/s 
     |████████████████████████████████| 271 kB 69.4 MB/s 
     |████████████████████████████████| 144 kB 67.2 MB/s 
     |████████████

In [2]:
paws = load_dataset('paws', 'labeled_final')
paws

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Dataset paws downloaded and prepared to /root/.cache/huggingface/datasets/paws/labeled_final/1.1.0/8d567c6472623f42bd2cc635cad06932d0f0cd2f897db56013c1180f4317d338. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
})

In [3]:
import csv
from pathlib import Path

colnames = 'sentence1 sentence2 label'.split()
filepath = Path('_'.join(colnames)).with_suffix('.csv')

with filepath.open('w') as fout:
    writer = csv.DictWriter(fout, fieldnames=colnames)
    writer.writeheader()
    for arrows in [paws['train'], paws['test'], paws['validation']]:
        for row in arrows:
            writer.writerow({name: row[name] for name in colnames})

In [4]:
quora = load_dataset('quora')

Using custom data configuration default


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

Dataset quora downloaded and prepared to /root/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
import pandas as pd
new_quora = {'sentence1':i['questions']['text'][0] for i in quora['train']}
new_quora['sentence2'] = [i['questions']['text'][1] for i in quora['train']]
new_quora['label'] = [0 if i['is_duplicate'] else 1 for i in quora['train']]
pd.DataFrame(new_quora).to_csv('quora_format_paws.csv')

In [6]:
paws = pd.read_csv('sentence1_sentence2_label.csv')
quora = pd.read_csv('quora_format_paws.csv')
df = pd.concat([paws, quora])
del paws
del quora
df.to_csv('data.csv')

In [48]:
df = pd.read_csv('data.csv').loc[:, ['sentence1', 'sentence2', 'label']]
x = df.drop(['sentence2'], axis=1).to_numpy()
y = list(df['sentence2'].to_numpy())
for i, j in enumerate(y):
  if not isinstance(j, str):
    del y[i]

In [49]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
new_x = [[i[0], i[1]] for i in zip(tokenizer(list(x[:,0]))['input_ids'], x[0,:])]
new_y = tokenizer(list(y))['input_ids']

In [52]:
import numpy as np
x = np.array(new_x)
y = np.array(new_y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
x

array([[list([0, 1121, 2201, 2156, 11, 779, 379, 2466, 2156, 37, 16340, 1145, 5, 2370, 6499, 2156, 13047, 2032, 9144, 22351, 261, 2156, 1996, 123, 13, 10, 12373, 7, 671, 7, 1156, 149, 3430, 479, 2]),
        'In Paris , in October 1560 , he secretly met the English ambassador , Nicolas Throckmorton , asking him for a passport to return to England through Scotland .'],
       [list([0, 133, 2762, 191, 9, 14873, 480, 5553, 21, 5, 389, 212, 191, 9, 5, 496, 12610, 1544, 479, 2]),
        0]], dtype=object)

In [14]:
model = TFAutoModel.from_pretrained("facebook/bart-large", forced_bos_token_id=0)

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBartModel.

All the layers of TFBartModel were initialized from the model checkpoint at facebook/bart-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartModel for predictions without further training.


In [15]:
model.compile(optimizer='Adam', loss='mse')

In [16]:
model.fit(x, y, epochs=10)
model.save('finetuned_bart.h5')

ValueError: ignored

In [17]:
x[0]

array([list([0, 1121, 2201, 2156, 11, 779, 379, 2466, 2156, 37, 16340, 1145, 5, 2370, 6499, 2156, 13047, 2032, 9144, 22351, 261, 2156, 1996, 123, 13, 10, 12373, 7, 671, 7, 1156, 149, 3430, 479, 2]),
       0], dtype=object)